In [1]:
#- Basic imports
import numpy as np
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [2]:
from astropy.table import Table
#- Read exposures
exposures = Table.read('exposures.fits')

In [3]:
from bokeh.models import ColumnDataSource
from astropy.time import Time
from bokeh.models import HoverTool, ColorBar
from bokeh.layouts import gridplot
from astropy.table import join
from astropy.time import TimezoneInfo
import astropy.units as u
from datetime import tzinfo
from datetime import datetime
from bokeh.models.glyphs import HBar
from bokeh.models import LabelSet, FactorRange
from bokeh.palettes import viridis
from bokeh.transform import factor_cmap
from bokeh.models.widgets.tables import DataTable, TableColumn
from astropy import coordinates

In [4]:
#- Separate calibration exposures
all_exposures = exposures[exposures['PROGRAM'] != 'CALIB']
all_calibs = exposures[exposures['PROGRAM'] == 'CALIB']

In [5]:
def getPhase(night):
    '''
    Returns the phase of the moon as a decimal given the number of nights since full_zero
    
    Args:
        night : number of nights since full_zero, an inital night of full moon
    '''
    phase = -0.5 * np.sin(np.pi/15 * (night)) + 0.5
    return phase


In [6]:
def phaseSign(night):
    dt = -1 * np.cos(np.pi/15*night)
    return dt


In [7]:
def toDatetime(night):
    '''
    Returns the night parsed into a datetime object
    
    Args:
        night: a single night given as YYYYMMDD in string representation
    '''
    n = datetime(int(night[:4]), int(night[4:6]), int(night[6:]))
    return n

In [8]:
def numNightsFullZero(date):
    '''
    Returns the number of nights since a specified night of the full moon,
    full_zero (11/12/2019 6:34)
    
    Args:
        night: a single night given as a datetime object
    '''
    full_zero = datetime(2019, 11, 12, 6, 34)
    diff = date - full_zero
    return diff.days



In [9]:
def getWedge(phase, der, fig, ra, dec):
    '''
    Given a phase of the moon, returns the bokeh rendered point to represent the phase on the skypath plot
    
    Args:
        phase: a proportion representing the phase of the moon 
            where 1 corresponds to full moon and 0 corresponds to no moon
    '''
    if (phase <= 0.25):
        return fig.arc(ra, dec, start_angle=0, end_angle=2*np.pi, radius=4, color='gold')
    elif (phase <= 0.75 and der > 0):
        return fig.wedge(ra, dec, start_angle=-np.pi/2, end_angle=np.pi/2, radius=4, color='gold')
    elif (phase <= 7.5 and der < 0):
        return fig.wedge(ra, dec, start_angle=np.pi/2, end_angle=3*np.pi/2, radius=4, color='gold')
    else:
        return fig.wedge(ra, dec, start_angle=0, end_angle=2*np.pi, radius=4, color='gold')

    

In [10]:
def getMoonRend(night, fig, ra, dec):
    n = numNightsFullZero(toDatetime(night))
    phase = getPhase(n)
    der = phaseSign(n)
    moon = getWedge(phase, der, fig, ra, dec)
    return moon


In [15]:
COLS = ['DATETIME', 'NIGHTSSINCE', 'PHASE']
dates = [toDatetime(n) for n in all_exposures['NIGHT']]
nightsSince = [numNightsFullZero(d) for d in dates]
phases = [getPhase(n) for n in nightsSince]

src = ColumnDataSource(data={'DATETIME': np.array(dates),
                             'NIGHTSSINCE': np.array(nightsSince),
                             'PHASE': np.array(phases)
                            })

fig = bk.figure(width=400, height=150)
a = fig.scatter(x='NIGHTSSINCE', y='PHASE', source=src)
fig.line(x='NIGHTSSINCE', y='PHASE', source=src)

TOOLTIPS = [("Date", "@DATETIME{%Y/%m/%d}"), ("PHASE", "@PHASE")]
hover = HoverTool(renderers = [a], tooltips=TOOLTIPS, formatters={"DATETIME":"datetime"})
fig.add_tools(hover)

bk.show(fig)